# Integration Examples: YOLOv8 Object Detection with Popular Frameworks

This notebook demonstrates how to integrate the YOLOv8 object detection functionality with popular deep learning frameworks and applications. We'll show examples of working with:

1. Hugging Face Spaces for web demo deployment
2. Integration with PyTorch Lightning for structured training
3. Working with different image formats and sources
4. Integration with computer vision pipelines

All examples ensure full compatibility with both Kaggle and Colab environments.

## 1. Setting up the Environment

First, let's set up the environment with all the necessary dependencies.

In [ ]:
# Clone the repository if running in Colab/Kaggle
import os
import sys

# Check if we're in Colab or Kaggle
IN_COLAB = 'google.colab' in sys.modules
IN_KAGGLE = 'kaggle_secrets' in sys.modules

# If we're in Colab or Kaggle, set up the environment
if IN_COLAB or IN_KAGGLE:
    # Clone the repository
    !git clone -q https://github.com/yourusername/object-detection-yolo.git
    %cd object-detection-yolo
    
    # Install dependencies
    !pip install -q ultralytics opencv-python ipywidgets matplotlib Pillow
    !pip install -q gradio pytorch-lightning albumentations
    
    # Add the repository root to the Python path
    sys.path.insert(0, os.getcwd())
    
    print(f"Setting up in {'Google Colab' if IN_COLAB else 'Kaggle'}")
else:
    print("Running locally")
    # Install additional dependencies if needed
    !pip install -q gradio pytorch-lightning albumentations

In [ ]:
# Import common libraries
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import urllib.request
import json
from PIL import Image
from io import BytesIO
import base64
import requests

# Import notebook utilities
try:
    from utils.notebook_utils import setup_env, download_sample_images
except ImportError:
    # Define minimal versions if not available
    def setup_env():
        print("Environment setup simplified due to import error")
    
    def download_sample_images(output_dir="sample_images"):
        os.makedirs(output_dir, exist_ok=True)
        samples = [
            {'url': 'https://ultralytics.com/images/zidane.jpg', 'name': 'person.jpg'},
            {'url': 'https://ultralytics.com/images/bus.jpg', 'name': 'bus.jpg'}
        ]
        paths = []
        for sample in samples:
            path = os.path.join(output_dir, sample['name'])
            if not os.path.exists(path):
                print(f"Downloading {sample['name']}...")
                urllib.request.urlretrieve(sample['url'], path)
            paths.append(path)
        return paths

# Setup environment
setup_env()

In [ ]:
# Import YOLOv8 detector
try:
    from src.yolo_detector import YOLOv8Detector
except ImportError:
    # If import fails, use the Ultralytics YOLO directly
    from ultralytics import YOLO
    
    class YOLOv8Detector:
        def __init__(self, model_size='n', conf=0.25, iou=0.45, device=None):
            if device is None:
                self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            else:
                self.device = device
                
            model_path = f'yolov8{model_size}.pt'
            self.model = YOLO(model_path)
            
            self.conf = conf
            self.iou = iou
            self.class_names = self.model.names
            
            print(f"YOLOv8{model_size} detector initialized on {self.device}")
        
        def detect(self, image, show_result=True, return_processed_image=False):
            result = self.model(image, conf=self.conf, iou=self.iou)[0]
            
            if show_result:
                im_array = result.plot()
                plt.figure(figsize=(12, 8))
                plt.imshow(cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB))
                plt.axis('off')
                
                if isinstance(image, str):
                    plt.title(f"Detection Results: {Path(image).name}")
                else:
                    plt.title(f"Detection Results")
                    
                plt.show()
                
                print(f"\nDetections:")
                boxes = result.boxes
                for i, box in enumerate(boxes):
                    class_id = int(box.cls.item())
                    class_name = self.class_names[class_id]
                    confidence = box.conf.item()
                    print(f"  {i+1}. {class_name} (Confidence: {confidence:.2f})")
            
            if return_processed_image:
                return result.plot()
            
            return result

# Initialize the YOLOv8 detector
detector = YOLOv8Detector(model_size='n', conf=0.25, iou=0.45)

## 2. Hugging Face Spaces Integration

In this section, we'll create a simple web demo using Gradio that can be deployed to Hugging Face Spaces. This allows anyone to use your object detection model through a web interface.

In [ ]:
# Import Gradio for web interface
import gradio as gr

In [ ]:
def process_image_for_detection(image, conf_threshold=0.25, iou_threshold=0.45, model_size='n'):
    """
    Process an image for object detection using YOLOv8.
    
    Args:
        image: Input image from Gradio (PIL Image)
        conf_threshold: Confidence threshold for detection
        iou_threshold: IoU threshold for NMS
        model_size: YOLOv8 model size ('n', 's', 'm', 'l', 'x')
        
    Returns:
        Annotated image with detections
    """
    try:
        # Convert PIL image to numpy array
        img_array = np.array(image)
        
        # Initialize detector with desired parameters
        detector = YOLOv8Detector(
            model_size=model_size,
            conf=conf_threshold,
            iou=iou_threshold
        )
        
        # Run detection
        result = detector.detect(img_array, show_result=False)
        
        # Get processed image with detections
        annotated_image = result.plot()
        
        # Get detection information
        boxes = result.boxes
        detection_info = ""
        
        if len(boxes) > 0:
            detection_info += f"Detected {len(boxes)} objects:\n"
            
            for i, box in enumerate(boxes):
                class_id = int(box.cls.item())
                class_name = detector.class_names[class_id]
                confidence = box.conf.item()
                detection_info += f"{i+1}. {class_name} (Confidence: {confidence:.2f})\n"
        else:
            detection_info = "No objects detected."
        
        # Convert BGR to RGB for displaying in Gradio
        annotated_image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
        
        return annotated_image_rgb, detection_info
    
    except Exception as e:
        return image, f"Error processing image: {str(e)}"

In [ ]:
# Create a Gradio interface
with gr.Blocks(title="YOLOv8 Object Detection") as demo:
    gr.Markdown("# YOLOv8 Object Detection Demo")
    gr.Markdown("Upload an image or take a photo to detect objects.")
    
    with gr.Row():
        with gr.Column():
            input_image = gr.Image(type="pil", label="Input Image")
            
            with gr.Row():
                conf_slider = gr.Slider(minimum=0.1, maximum=0.9, value=0.25, step=0.05, label="Confidence Threshold")
                iou_slider = gr.Slider(minimum=0.1, maximum=0.9, value=0.45, step=0.05, label="IoU Threshold")
            
            model_dropdown = gr.Dropdown(
                choices=["n", "s", "m", "l", "x"],
                value="n",
                label="Model Size",
                info="n=nano, s=small, m=medium, l=large, x=xlarge"
            )
            
            detect_button = gr.Button("Detect Objects")
        
        with gr.Column():
            output_image = gr.Image(type="numpy", label="Detection Results")
            output_text = gr.Textbox(label="Detection Information", lines=8)
    
    # Add examples
    gr.Examples(
        examples=["https://ultralytics.com/images/zidane.jpg", "https://ultralytics.com/images/bus.jpg"],
        inputs=input_image,
    )
    
    # Set up button click event
    detect_button.click(
        fn=process_image_for_detection,
        inputs=[input_image, conf_slider, iou_slider, model_dropdown],
        outputs=[output_image, output_text]
    )
    
    gr.Markdown(
        """
        ### How to Use
        1. Upload an image or take a photo using your webcam
        2. Adjust the confidence and IoU thresholds if needed
        3. Select a model size (larger models are more accurate but slower)
        4. Click "Detect Objects" to run the detection
        
        ### About
        This demo uses YOLOv8, a state-of-the-art object detection model that can detect 80 different object categories.
        """
    )

# Launch the demo
demo.launch(share=True, inline=True)

### Deploying to Hugging Face Spaces

To deploy this demo to Hugging Face Spaces, you'll need to create the following files:

1. `app.py` with the Gradio code
2. `requirements.txt` with the dependencies
3. Any supporting modules

Let's create these files:

In [ ]:
# Create app.py
app_py = """
import gradio as gr
import numpy as np
import cv2
import torch
from ultralytics import YOLO

# Helper class for YOLOv8 detection
class YOLOv8Detector:
    def __init__(self, model_size='n', conf=0.25, iou=0.45, device=None):
        if device is None:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        else:
            self.device = device
            
        model_path = f'yolov8{model_size}.pt'
        self.model = YOLO(model_path)
        
        self.conf = conf
        self.iou = iou
        self.class_names = self.model.names
        
        print(f"YOLOv8{model_size} detector initialized on {self.device}")
    
    def detect(self, image, show_result=False):
        result = self.model(image, conf=self.conf, iou=self.iou)[0]
        return result

def process_image_for_detection(image, conf_threshold=0.25, iou_threshold=0.45, model_size='n'):
    """Process an image for object detection using YOLOv8."""
    try:
        # Convert PIL image to numpy array
        img_array = np.array(image)
        
        # Initialize detector with desired parameters
        detector = YOLOv8Detector(
            model_size=model_size,
            conf=conf_threshold,
            iou=iou_threshold
        )
        
        # Run detection
        result = detector.detect(img_array)
        
        # Get processed image with detections
        annotated_image = result.plot()
        
        # Get detection information
        boxes = result.boxes
        detection_info = ""
        
        if len(boxes) > 0:
            detection_info += f"Detected {len(boxes)} objects:\n"
            
            for i, box in enumerate(boxes):
                class_id = int(box.cls.item())
                class_name = detector.class_names[class_id]
                confidence = box.conf.item()
                detection_info += f"{i+1}. {class_name} (Confidence: {confidence:.2f})\n"
        else:
            detection_info = "No objects detected."
        
        # Convert BGR to RGB for displaying in Gradio
        annotated_image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
        
        return annotated_image_rgb, detection_info
    
    except Exception as e:
        return image, f"Error processing image: {str(e)}"

# Create a Gradio interface
with gr.Blocks(title="YOLOv8 Object Detection") as demo:
    gr.Markdown("# YOLOv8 Object Detection Demo")
    gr.Markdown("Upload an image or take a photo to detect objects.")
    
    with gr.Row():
        with gr.Column():
            input_image = gr.Image(type="pil", label="Input Image")
            
            with gr.Row():
                conf_slider = gr.Slider(minimum=0.1, maximum=0.9, value=0.25, step=0.05, label="Confidence Threshold")
                iou_slider = gr.Slider(minimum=0.1, maximum=0.9, value=0.45, step=0.05, label="IoU Threshold")
            
            model_dropdown = gr.Dropdown(
                choices=["n", "s", "m", "l", "x"],
                value="n",
                label="Model Size",
                info="n=nano, s=small, m=medium, l=large, x=xlarge"
            )
            
            detect_button = gr.Button("Detect Objects")
        
        with gr.Column():
            output_image = gr.Image(type="numpy", label="Detection Results")
            output_text = gr.Textbox(label="Detection Information", lines=8)
    
    # Add examples
    gr.Examples(
        examples=["https://ultralytics.com/images/zidane.jpg", "https://ultralytics.com/images/bus.jpg"],
        inputs=input_image,
    )
    
    # Set up button click event
    detect_button.click(
        fn=process_image_for_detection,
        inputs=[input_image, conf_slider, iou_slider, model_dropdown],
        outputs=[output_image, output_text]
    )
    
    gr.Markdown(
        """
        ### How to Use
        1. Upload an image or take a photo using your webcam
        2. Adjust the confidence and IoU thresholds if needed
        3. Select a model size (larger models are more accurate but slower)
        4. Click "Detect Objects" to run the detection
        
        ### About
        This demo uses YOLOv8, a state-of-the-art object detection model that can detect 80 different object categories.
        """
    )

# Launch the demo
demo.launch()
"""

# Create requirements.txt
requirements_txt = """
ultralytics>=8.0.0
gradio>=3.50.0
torch>=1.7.0
opencv-python>=4.5.0
numpy>=1.20.0
Pillow>=8.0.0
"""

# Create README.md
readme_md = """
# YOLOv8 Object Detection Demo

This Space demonstrates object detection using YOLOv8, a state-of-the-art object detection model.

## How to Use

1. Upload an image or take a photo using your webcam
2. Adjust the confidence and IoU thresholds if needed
3. Select a model size (larger models are more accurate but slower)
4. Click "Detect Objects" to run the detection

## About YOLOv8

YOLOv8 is the latest version in the YOLO (You Only Look Once) family of models. It can detect 80 different object categories with high accuracy and speed.

## Model Sizes

- **YOLOv8n**: Nano model (smallest, fastest, least accurate)
- **YOLOv8s**: Small model
- **YOLOv8m**: Medium model
- **YOLOv8l**: Large model
- **YOLOv8x**: Extra-large model (largest, slowest, most accurate)
"""

# Save these files to disk
os.makedirs('huggingface_demo', exist_ok=True)

with open('huggingface_demo/app.py', 'w') as f:
    f.write(app_py)

with open('huggingface_demo/requirements.txt', 'w') as f:
    f.write(requirements_txt)

with open('huggingface_demo/README.md', 'w') as f:
    f.write(readme_md)

print("Created files for Hugging Face Spaces deployment in 'huggingface_demo' directory")

## 3. Integration with PyTorch Lightning

PyTorch Lightning provides a more structured way to organize PyTorch code. Here's how to integrate YOLOv8 with PyTorch Lightning for a more organized approach to object detection.

In [ ]:
# Import PyTorch Lightning
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import Dataset, DataLoader

# Create a PyTorch Lightning module for YOLOv8
class YOLOv8Module(pl.LightningModule):
    def __init__(self, model_size='n', conf=0.25, iou=0.45):
        super().__init__()
        self.save_hyperparameters()
        
        # Initialize YOLOv8 detector
        self.detector = YOLOv8Detector(
            model_size=model_size,
            conf=conf,
            iou=iou
        )
        
        # Store the class names
        self.class_names = self.detector.class_names
    
    def forward(self, x):
        # Forward pass through the model
        return self.detector.model(x)
    
    def detect_objects(self, image):
        # Detect objects in an image
        return self.detector.detect(image, show_result=False)
    
    def configure_optimizers(self):
        # This would be used for training
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        return optimizer
    
    def predict_step(self, batch, batch_idx):
        # Process a batch of images
        return self(batch)

# Create a simple dataset for object detection
class ObjectDetectionDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # Load image
        img_path = self.image_paths[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Apply transforms if any
        if self.transform:
            img = self.transform(image=img)['image']
        
        return img, img_path

In [ ]:
# Import albumentations for data augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Create a transform for preprocessing images
transform = A.Compose([
    A.Resize(640, 640),  # Resize to YOLOv8 input size
    A.Normalize(mean=[0, 0, 0], std=[1, 1, 1]),  # Normalize for YOLOv8
    ToTensorV2(),  # Convert to tensor
])

# Download sample images
image_paths = download_sample_images()

# Create a dataset
dataset = ObjectDetectionDataset(image_paths, transform=transform)

# Create a dataloader
dataloader = DataLoader(dataset, batch_size=2, shuffle=False, num_workers=0)

# Initialize the PyTorch Lightning module
model = YOLOv8Module(model_size='n', conf=0.25, iou=0.45)

# Create a trainer
trainer = pl.Trainer(max_epochs=1, accelerator='auto')

# Show how to predict with the trainer
print("\nPredicting with PyTorch Lightning:")
predictions = trainer.predict(model, dataloader)
print(f"Received {len(predictions)} batches of predictions")

In [ ]:
# Now let's use our YOLOv8Module to detect objects in images
for image_path in image_paths:
    print(f"\nDetecting objects in {image_path}")
    result = model.detect_objects(image_path)
    
    # Display the result
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(result.plot(), cv2.COLOR_BGR2RGB))
    plt.title(f"Detection Results: {Path(image_path).name}")
    plt.axis('off')
    plt.show()
    
    # Print detection results
    boxes = result.boxes
    print(f"Detected {len(boxes)} objects:")
    for i, box in enumerate(boxes):
        class_id = int(box.cls.item())
        class_name = model.class_names[class_id]
        confidence = box.conf.item()
        print(f"  {i+1}. {class_name} (Confidence: {confidence:.2f})")

## 4. Working with Different Image Formats and Sources

YOLOv8 can work with various image formats and sources. Let's explore how to handle different types of inputs.

In [ ]:
def detect_from_various_sources(detector, sources):
    """Demonstrate object detection from various image sources."""
    results = []
    
    for source in sources:
        print(f"\nProcessing source: {source['type']}")
        
        # Process based on source type
        if source['type'] == 'file':
            # Local file
            image_path = source['path']
            result = detector.detect(image_path, show_result=False)
            
            # Display the result
            plt.figure(figsize=(12, 8))
            plt.imshow(cv2.cvtColor(result.plot(), cv2.COLOR_BGR2RGB))
            plt.title(f"File: {Path(image_path).name}")
            plt.axis('off')
            plt.show()
            
        elif source['type'] == 'url':
            # Image from URL
            url = source['url']
            try:
                # Download image
                response = requests.get(url)
                img = Image.open(BytesIO(response.content))
                img_array = np.array(img)
                
                # Convert RGB to BGR for OpenCV compatibility
                if len(img_array.shape) == 3 and img_array.shape[2] == 3:
                    img_array = img_array[:, :, ::-1].copy()
                
                # Detect objects
                result = detector.detect(img_array, show_result=False)
                
                # Display the result
                plt.figure(figsize=(12, 8))
                plt.imshow(cv2.cvtColor(result.plot(), cv2.COLOR_BGR2RGB))
                plt.title(f"URL: {url.split('/')[-1]}")
                plt.axis('off')
                plt.show()
                
            except Exception as e:
                print(f"Error processing URL {url}: {e}")
                continue
        
        elif source['type'] == 'base64':
            # Base64 encoded image
            base64_data = source['data']
            try:
                # Decode base64 data
                img_data = base64.b64decode(base64_data)
                img = Image.open(BytesIO(img_data))
                img_array = np.array(img)
                
                # Convert RGB to BGR for OpenCV compatibility
                if len(img_array.shape) == 3 and img_array.shape[2] == 3:
                    img_array = img_array[:, :, ::-1].copy()
                
                # Detect objects
                result = detector.detect(img_array, show_result=False)
                
                # Display the result
                plt.figure(figsize=(12, 8))
                plt.imshow(cv2.cvtColor(result.plot(), cv2.COLOR_BGR2RGB))
                plt.title("Base64 Image")
                plt.axis('off')
                plt.show()
                
            except Exception as e:
                print(f"Error processing base64 image: {e}")
                continue
        
        elif source['type'] == 'numpy':
            # Numpy array
            img_array = source['array']
            
            # Detect objects
            result = detector.detect(img_array, show_result=False)
            
            # Display the result
            plt.figure(figsize=(12, 8))
            plt.imshow(cv2.cvtColor(result.plot(), cv2.COLOR_BGR2RGB))
            plt.title("Numpy Array Image")
            plt.axis('off')
            plt.show()
        
        else:
            print(f"Unknown source type: {source['type']}")
            continue
        
        # Print detection summary
        boxes = result.boxes
        print(f"Detected {len(boxes)} objects:")
        for i, box in enumerate(boxes):
            class_id = int(box.cls.item())
            class_name = detector.class_names[class_id]
            confidence = box.conf.item()
            print(f"  {i+1}. {class_name} (Confidence: {confidence:.2f})")
        
        results.append(result)
    
    return results

In [ ]:
# Create sample sources
sources = []

# 1. Local file
if image_paths:
    sources.append({
        'type': 'file',
        'path': image_paths[0]
    })

# 2. URL
sources.append({
    'type': 'url',
    'url': 'https://raw.githubusercontent.com/ultralytics/assets/main/im/image3.jpg'
})

# 3. Numpy array (create a simple test image)
test_image = np.zeros((300, 300, 3), dtype=np.uint8)
test_image[50:250, 50:250, :] = [0, 0, 255]  # Draw a red square
sources.append({
    'type': 'numpy',
    'array': test_image
})

# Process different sources
detect_from_various_sources(detector, sources)

## 5. Integration with Computer Vision Pipelines

YOLOv8 can be integrated into larger computer vision pipelines. Here's an example of combining object detection with additional processing steps.

In [ ]:
class ComputerVisionPipeline:
    """A pipeline that combines object detection with other CV tasks."""
    
    def __init__(self, detector=None):
        """Initialize the pipeline."""
        # Initialize the object detector
        if detector is None:
            self.detector = YOLOv8Detector(model_size='n', conf=0.25, iou=0.45)
        else:
            self.detector = detector
    
    def preprocess_image(self, image):
        """Preprocess the input image."""
        # Convert to numpy array if needed
        if isinstance(image, str):
            # Load from file path
            img = cv2.imread(image)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        elif isinstance(image, Image.Image):
            # Convert PIL image to numpy array
            img = np.array(image)
        else:
            # Assume it's already a numpy array
            img = image.copy()
        
        # Ensure image is in RGB format
        if len(img.shape) == 3 and img.shape[2] == 3:
            if np.max(img) <= 1.0:
                img = (img * 255).astype(np.uint8)  # Normalize to 0-255 if needed
        
        return img
    
    def detect_objects(self, image):
        """Detect objects in the image."""
        return self.detector.detect(image, show_result=False)
    
    def process_detections(self, image, result):
        """Process detection results for further analysis."""
        # Get bounding boxes
        boxes = result.boxes
        
        # Initialize results dictionary
        processed_results = {
            'objects': [],
            'object_count': len(boxes),
            'class_counts': {},
            'largest_object': None,
            'average_confidence': 0.0
        }
        
        # Extract detection information
        confidences = []
        largest_area = 0
        largest_object_idx = -1
        
        for i, box in enumerate(boxes):
            # Get class information
            class_id = int(box.cls.item())
            class_name = self.detector.class_names[class_id]
            confidence = float(box.conf.item())
            
            # Get bounding box coordinates
            bbox = box.xyxy[0].tolist()  # xyxy format is [x1, y1, x2, y2]
            x1, y1, x2, y2 = bbox
            width = x2 - x1
            height = y2 - y1
            area = width * height
            
            # Check if this is the largest object
            if area > largest_area:
                largest_area = area
                largest_object_idx = i
            
            # Extract object region of interest
            roi = image[int(y1):int(y2), int(x1):int(x2)].copy()
            
            # Add to object list
            processed_results['objects'].append({
                'class_id': class_id,
                'class_name': class_name,
                'confidence': confidence,
                'bbox': bbox,
                'area': area,
                'roi': roi
            })
            
            # Update class counts
            if class_name in processed_results['class_counts']:
                processed_results['class_counts'][class_name] += 1
            else:
                processed_results['class_counts'][class_name] = 1
            
            # Add to confidence list
            confidences.append(confidence)
        
        # Set largest object
        if largest_object_idx >= 0:
            processed_results['largest_object'] = processed_results['objects'][largest_object_idx]
        
        # Calculate average confidence
        if confidences:
            processed_results['average_confidence'] = sum(confidences) / len(confidences)
        
        return processed_results
    
    def visualize_results(self, image, processed_results):
        """Visualize the processed results."""
        # Create a copy of the image for visualization
        vis_image = image.copy()
        
        # Draw bounding boxes and labels for all objects
        for obj in processed_results['objects']:
            # Get bounding box coordinates
            x1, y1, x2, y2 = obj['bbox']
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            
            # Get class name and confidence
            class_name = obj['class_name']
            confidence = obj['confidence']
            
            # Draw bounding box
            cv2.rectangle(vis_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Draw label background
            text = f"{class_name} {confidence:.2f}"
            text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
            cv2.rectangle(vis_image, (x1, y1 - text_size[1] - 5), (x1 + text_size[0], y1), (0, 255, 0), -1)
            
            # Draw label text
            cv2.putText(vis_image, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
        
        # Draw summary information
        summary_text = [
            f"Total Objects: {processed_results['object_count']}",
            f"Avg Confidence: {processed_results['average_confidence']:.2f}"
        ]
        
        for i, text in enumerate(summary_text):
            y_pos = 30 + i * 30
            cv2.putText(vis_image, text, (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        
        return vis_image
    
    def analyze_scene(self, image):
        """Analyze the scene for objects and their relationships."""
        # Preprocess the image
        processed_image = self.preprocess_image(image)
        
        # Detect objects
        detection_result = self.detect_objects(processed_image)
        
        # Process detections
        processed_results = self.process_detections(processed_image, detection_result)
        
        # Visualize results
        visualization = self.visualize_results(processed_image, processed_results)
        
        # Display results
        plt.figure(figsize=(12, 8))
        plt.imshow(visualization)
        plt.title("Scene Analysis Results")
        plt.axis('off')
        plt.show()
        
        # Print analysis summary
        print("\nScene Analysis Summary:")
        print(f"Total objects detected: {processed_results['object_count']}")
        print(f"Average confidence: {processed_results['average_confidence']:.2f}")
        
        print("\nObject classes:")
        for class_name, count in processed_results['class_counts'].items():
            print(f"  {class_name}: {count}")
        
        if processed_results['largest_object']:
            largest = processed_results['largest_object']
            print(f"\nLargest object: {largest['class_name']} (Area: {largest['area']:.1f} pixels²)")
            
            # Display the largest object ROI
            plt.figure(figsize=(6, 6))
            plt.imshow(largest['roi'])
            plt.title(f"Largest Object: {largest['class_name']}")
            plt.axis('off')
            plt.show()
        
        return processed_results, visualization

In [ ]:
# Create a computer vision pipeline
pipeline = ComputerVisionPipeline(detector)

# Test the pipeline on a sample image
if image_paths:
    sample_image = image_paths[0]
    print(f"\nAnalyzing scene in {sample_image}...")
    processed_results, visualization = pipeline.analyze_scene(sample_image)

## 6. Summary and Next Steps

In this notebook, we've demonstrated several ways to integrate YOLOv8 object detection functionality with popular frameworks and applications:

1. **Hugging Face Spaces Integration**: We created a web demo using Gradio that can be deployed to Hugging Face Spaces, allowing anyone to use the model through a web interface.

2. **PyTorch Lightning Integration**: We showed how to integrate YOLOv8 with PyTorch Lightning for a more structured approach to object detection, which would be particularly useful for training and fine-tuning workflows.

3. **Working with Different Image Formats**: We demonstrated how to handle various image sources, including local files, URLs, base64-encoded images, and numpy arrays.

4. **Computer Vision Pipelines**: We created a comprehensive pipeline that combines object detection with additional analysis and visualization steps.

### Next Steps

In the next notebook, we'll explore real-time object detection with a webcam feed, which will complete our set of core functionalities for the YOLOv8 object detection project.